In [2]:
import json

with open("/content/domain_knowledge.json", "r") as json_file:
    fertilizer_data = json.load(json_file)

print(fertilizer_data)


{'soil_preferences': {'Banana': {'pH_range': [5.5, 7.0], 'suitable_soils': ['Loamy', 'Clay']}, 'Coffee': {'pH_range': [5.0, 6.5], 'suitable_soils': ['Loamy', 'Sandy']}, 'Rice': {'pH_range': [5.5, 7.5], 'suitable_soils': ['Clay', 'Loamy']}}, 'plant_conditions': {'Banana': {'temp_range': [18, 30], 'humidity_range': [50, 80]}, 'Coffee': {'temp_range': [15, 28], 'humidity_range': [60, 90]}, 'Rice': {'temp_range': [20, 35], 'humidity_range': [70, 100]}}, 'nutrient_requirements': {'Banana': {'N': 200, 'P': 60, 'K': 350}, 'Coffee': {'N': 150, 'P': 50, 'K': 200}, 'Rice': {'N': 100, 'P': 40, 'K': 150}}, 'fertilizer_recommendations': {'Nitrogen': {'Mild': 'Apply 20-30 kg/ha of urea as a foliar spray.', 'Moderate': 'Apply 50 kg/ha of urea in split doses.', 'Severe': 'Apply 80-100 kg/ha of urea with soil incorporation.'}, 'Phosphorus': {'Mild': 'Apply 15-25 kg/ha of superphosphate.', 'Moderate': 'Apply 40 kg/ha of superphosphate.', 'Severe': 'Apply 60 kg/ha of superphosphate, deep placement.'}, 'P

In [3]:
from abc import ABC, abstractmethod
import json

# Load domain knowledge from JSON file
with open('domain_knowledge.json', 'r') as f:
    DOMAIN_KNOWLEDGE = json.load(f)

# Abstract Base Classes
class SoilBase(ABC):
    def __init__(self, soil_type, pH):
        self.soil_type = soil_type
        self.pH = pH

    @abstractmethod
    def get_soil_type(self):
        pass

    @abstractmethod
    def get_soil_nutrient(self, nutrient):
        pass

    @abstractmethod
    def is_suitable_for_plant(self, plant_name):
        pass

class EnvironmentBase(ABC):
    def __init__(self, temperature, humidity):
        self.temperature = temperature
        self.humidity = humidity

    @abstractmethod
    def get_temperature(self):
        pass

    @abstractmethod
    def get_humidity(self):
        pass

    @abstractmethod
    def check_suitability(self, plant_type):
        pass

class PlantBase(ABC):
    @abstractmethod
    def get_name(self):
        pass

    @abstractmethod
    def get_required_nutrients(self):
        pass

    @abstractmethod
    def check_growth_conditions(self, environment):
        pass

class DeficiencyBase(ABC):
    @abstractmethod
    def get_deficiency_name(self):
        pass

    @abstractmethod
    def assess_severity(self, symptoms):
        pass

    @abstractmethod
    def recommend_treatment(self, severity):
        pass

class FertilizerStrategyBase(ABC):
    @abstractmethod
    def get_fertilizer_recommendation(self, severity):
        pass

    @abstractmethod
    def get_application_guidelines(self):
        pass

# Concrete Classes
class Soil(SoilBase):
    def __init__(self, soil_type, pH, nutrient_levels):
        super().__init__(soil_type, pH)
        self.nutrient_levels = nutrient_levels

    def get_soil_type(self):
        return self.soil_type

    def is_suitable_for_plant(self, plant_name):
        soil_preferences = DOMAIN_KNOWLEDGE["soil_preferences"]
        if plant_name in soil_preferences:
            prefs = soil_preferences[plant_name]
            if prefs["pH_range"][0] <= self.pH <= prefs["pH_range"][1] and self.soil_type in prefs["suitable_soils"]:
                return f"✅ Suitable: {self.soil_type} soil with pH {self.pH} is ideal for {plant_name}."
            else:
                return f"⚠️ Not Suitable: {plant_name} prefers {prefs['suitable_soils']} soil with pH between {prefs['pH_range'][0]} and {prefs['pH_range'][1]}, but got {self.soil_type} with pH {self.pH}."
        return "❌ Unknown plant type. Cannot determine soil suitability."

    def get_soil_nutrient(self, nutrient):
        level = self.nutrient_levels.get(nutrient)
        if level is not None:
            return f"✅ {nutrient} level in soil: {level} ppm."
        else:
            return f"⚠️ {nutrient} not found in soil data."

class Environment(EnvironmentBase):
    def __init__(self, temperature, humidity):
        super().__init__(temperature, humidity)

    def get_temperature(self):
        return self.temperature

    def get_humidity(self):
        return self.humidity

    def check_suitability(self, plant_type):
        plant_conditions = DOMAIN_KNOWLEDGE["plant_conditions"]
        if plant_type in plant_conditions:
            conditions = plant_conditions[plant_type]
            temp_ok = conditions["temp_range"][0] <= self.temperature <= conditions["temp_range"][1]
            humidity_ok = conditions["humidity_range"][0] <= self.humidity <= conditions["humidity_range"][1]
            if temp_ok and humidity_ok:
                return f"✅ Environment suitable: Temperature {self.temperature}°C & Humidity {self.humidity}% are within optimal range for {plant_type}."
            else:
                return f"⚠️ Not Suitable: {plant_type} requires temperature {conditions['temp_range']}°C & humidity {conditions['humidity_range']}%, but got {self.temperature}°C and {self.humidity}%."
        return "❌ Unknown plant type. Cannot determine environmental suitability."

class Plant(PlantBase):
    def __init__(self, name, soil, environment):
        self.name = name
        self.soil = soil
        self.environment = environment

    def get_name(self):
        return self.name

    def get_required_nutrients(self):
        return DOMAIN_KNOWLEDGE["nutrient_requirements"].get(self.name, {})

    def check_growth_conditions(self, environment):
        return self.environment.check_suitability(self.name)

class BananaPlant(Plant):
    def __init__(self, soil, environment):
        super().__init__("Banana", soil, environment)

class CoffeePlant(Plant):
    def __init__(self, soil, environment):
        super().__init__("Coffee", soil, environment)

class RicePlant(Plant):
    def __init__(self, soil, environment):
        super().__init__("Rice", soil, environment)

class DeficiencyEvaluator:
    def __init__(self, deficiency_type, severity_level):
        self.deficiency_type = deficiency_type
        self.severity_level = severity_level

    def evaluate_deficiency(self):
        recommendations = DOMAIN_KNOWLEDGE["fertilizer_recommendations"]
        recommendation_text = recommendations.get(self.deficiency_type, {}).get(self.severity_level, "No specific recommendation available.")
        return {
            "Deficiency": self.deficiency_type,
            "Severity": self.severity_level,
            "Recommendation": recommendation_text
        }

class FertilizerStrategyLow(FertilizerStrategyBase):
    def get_fertilizer_recommendation(self, severity):
        return DOMAIN_KNOWLEDGE["fertilizer_recommendations"]

    def get_application_guidelines(self):
        return DOMAIN_KNOWLEDGE["fertilizer_application_guidelines"]

    def get_recommendation(self, deficiency_type):
        return DOMAIN_KNOWLEDGE["fertilizer_recommendations"].get(deficiency_type, {}).get("Mild", "No recommendation available for this deficiency.")

class FertilizerStrategyMedium(FertilizerStrategyBase):
    def get_fertilizer_recommendation(self, severity):
        return DOMAIN_KNOWLEDGE["fertilizer_recommendations"]

    def get_application_guidelines(self):
        return DOMAIN_KNOWLEDGE["fertilizer_application_guidelines"]

    def get_recommendation(self, deficiency_type):
        return DOMAIN_KNOWLEDGE["fertilizer_recommendations"].get(deficiency_type, {}).get("Moderate", "No recommendation available for this deficiency.")

class FertilizerStrategyHigh(FertilizerStrategyBase):
    def get_fertilizer_recommendation(self, severity):
        return DOMAIN_KNOWLEDGE["fertilizer_recommendations"]

    def get_application_guidelines(self):
        return DOMAIN_KNOWLEDGE["fertilizer_application_guidelines"]

    def get_recommendation(self, deficiency_type):
        return DOMAIN_KNOWLEDGE["fertilizer_recommendations"].get(deficiency_type, {}).get("Severe", "No recommendation available for this deficiency.")

class FertilizerPipeline:
    def __init__(self):
        self.recommendation_engine = RecommendationEngine()

    def run(self, soil_type, plant_type, deficiency_type, severity_level, temperature, humidity):
        pH = 6.5
        nutrient_levels = {"N": 40, "P": 30, "K": 50}
        soil = Soil(soil_type, pH, nutrient_levels)
        environment = Environment(temperature, humidity)

        soil_suitability = soil.is_suitable_for_plant(plant_type)
        environment_status = environment.check_suitability(plant_type)

        recommendation = self.recommendation_engine.get_fertilizer_recommendation(
            plant=plant_type,
            deficiency=deficiency_type,
            severity=severity_level,
            soil_suitability=soil_suitability,
            temperature_status=environment_status,
            humidity_status=environment_status
        )
        return recommendation

class RecommendationEngine:
    def __init__(self):
        self.strategies = {
            "Mild": FertilizerStrategyLow(),
            "Moderate": FertilizerStrategyMedium(),
            "Severe": FertilizerStrategyHigh()
        }

    def get_fertilizer_recommendation(self, plant, deficiency, severity, soil_suitability, temperature_status, humidity_status):
        strategy = self.strategies.get(severity, None)
        if not strategy:
            return f"Error: Invalid severity level '{severity}'. Expected: Mild, Moderate, or Severe."

        recommendation = strategy.get_recommendation(deficiency)
        guidelines = strategy.get_application_guidelines()

        output_message = f"""
        === Fertilizer Recommendations ===
        Plant: {plant}
        Deficiency: {deficiency}
        Severity: {severity}
        Recommendation: {recommendation}

        === Additional Details ===
        Soil Suitability: {soil_suitability}
        Temperature Status: {temperature_status}
        Humidity Status: {humidity_status}

        #### **Fertilizer Application Guidelines (Applicable to Banana, Coffee & Rice)**
        - **Application Methods:**
          - **Even & Precise Spreading:** {guidelines['Application_Methods']['Even_Precise_Spreading']}
          - **Split Applications:**
            - **Nitrogen:** {guidelines['Application_Methods']['Split_Applications']['Nitrogen']}
            - **Phosphorus, Potassium, Zinc:** {guidelines['Application_Methods']['Split_Applications']['Phosphorus_Potassium_Zinc']}
          - **Equipment & Conditions:** {guidelines['Application_Methods']['Equipment_Conditions']}

        - **General Best Practices:**
          - **Quality & Consistency:** {guidelines['General_Best_Practices']['Quality_Consistency']}
          - **Timing & Precision:** {guidelines['General_Best_Practices']['Timing_Precision']}
          - **Zone-Specific Recommendations:** {guidelines['General_Best_Practices']['Zone_Specific_Recommendations']}
        """
        return output_message.strip()

# Main execution
pipeline = FertilizerPipeline()
plant_deficiency_map = DOMAIN_KNOWLEDGE["plant_deficiency_map"]
severity_levels = DOMAIN_KNOWLEDGE["severity_levels"]

for plant, deficiencies in plant_deficiency_map.items():
    for deficiency in deficiencies:
        for severity in severity_levels:
            print("\n" + "="*50)
            print(f"Generating Fertilizer Recommendation for {plant} | {deficiency} | {severity}\n")
            result = pipeline.run(
                soil_type="Loamy",
                plant_type=plant,
                deficiency_type=deficiency,
                severity_level=severity,
                temperature=25,
                humidity=60
            )
            print(result)
            print("="*50 + "\n")


Generating Fertilizer Recommendation for Banana | Magnesium | Mild

=== Fertilizer Recommendations ===
        Plant: Banana
        Deficiency: Magnesium
        Severity: Mild
        Recommendation: Foliar spray of 5% MgSO4 or dolomite limestone at 3 t/ha.

        === Additional Details ===
        Soil Suitability: ✅ Suitable: Loamy soil with pH 6.5 is ideal for Banana.
        Temperature Status: ✅ Environment suitable: Temperature 25°C & Humidity 60% are within optimal range for Banana.
        Humidity Status: ✅ Environment suitable: Temperature 25°C & Humidity 60% are within optimal range for Banana.

        #### **Fertilizer Application Guidelines (Applicable to Banana, Coffee & Rice)**
        - **Application Methods:**
          - **Even & Precise Spreading:** Use calibrated spreaders and perform tray tests to ensure even distribution.
          - **Split Applications:**
            - **Nitrogen:** Apply in multiple splits—50% before transplanting, 25% at 30 days, 25% at 